In [ ]:
import glob
import io
import numpy as np
import os
import pandas as pd
import PyPDF2
import re
import shutil
import sys
import traceback
import threading
import warnings
import win32com.client as win32

from openpyxl import load_workbook, Workbook
from openpyxl.styles import Alignment, Border, Font, NamedStyle, PatternFill, Side
from openpyxl.utils import get_column_letter, range_boundaries
from openpyxl.cell.cell import MergedCell
from pandas import Timestamp
from pathlib import Path
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from tqdm import tqdm

from src.anexo_b.process_b_attachment import anexo_b
from src.anexo_b.normalize_text import format_values

In [4]:
anexo_b = anexo_b()

In [ ]:
dir_Arquivos = Path(
    "C:/Users/jaquelinerufino/OneDrive - KPMG/Documents/Desenvolvimento/Lighthouse/Itaú/01. Originais/"
)
dir_ContratosFinais = Path(
    "C:/Users/jaquelinerufino/OneDrive - KPMG/Documents/Desenvolvimento/Lighthouse/Itaú/05. Output/Contratos Finais/2018/"
)
output_pdf = Path("C:/Anexo N_C/pdf")
output_excel = Path("C:/Prioritários/150 faltantes/Excel/")

status_cols = [
    "Contrato",
    "Cliente - Status",
    "CNPJ/CPF - Status",
    "Valor - Status",
    "Data do Contrato - Status",
    "Data de Liquidação - Status",
    "Quantidade de Parcelas - Status",
]

In [ ]:
itens = pd.read_csv(
    Path.joinpath(dir_Arquivos, "150 Contratos.txt"),
    dtype=str,
    names=["Contrato"],
    skiprows=1,
)
comparativo = pd.read_excel(
    Path.joinpath(dir_Arquivos, "Status_2018.xlsx"), dtype={"Contrato": str}
)
caminho_base = Path.joinpath(dir_Arquivos, "Modelo_Contrato_03.xlsx")
path_base = Path.joinpath(dir_Arquivos, "DOC_BASE_Reprocessado (1).xlsx")

In [ ]:
df_contratuais_vs_registro_contabil = pd.read_csv(
    Path.joinpath(
        dir_ContratosFinais, "Quadro 1 - Dados Contratuais x Registro Contábil.csv"
    ),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_demonstracao_valor_anual = pd.read_csv(
    Path.joinpath(dir_ContratosFinais, "Quadro 2 - Superveniência.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_efeito_resultado_contabil = pd.read_csv(
    Path.joinpath(dir_ContratosFinais, "Quadro 3 - Resultado Contábil.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_base_calculo_irpj = pd.read_csv(
    Path.joinpath(dir_ContratosFinais, "Quadro 4 - IRPJ.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_base_calculo_csll = pd.read_csv(
    Path.joinpath(dir_ContratosFinais, "Quadro 5 - CSSL.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_demonstrativo_efeito_nulo = pd.read_csv(
    Path.joinpath(
        dir_ContratosFinais,
        "Quadro 6 - Efeito Nulo da Superveniência_Insuficiência.csv",
    ),
    encoding="latin1",
    sep=",",
    dtype=str,
)

In [ ]:
lst_itens = itens["Contrato"].tolist()

comparativo["Contrato"] = comparativo["Contrato"].astype(str)
comparativo = comparativo[comparativo["Contrato"].isin(lst_itens)]

## Definitions

### Processamento dos Quadros

#### processar_quadro_zero

nao precisa mexer

In [ ]:
def processar_quadro_zero(df_quadro_zero, status_cols, contrato, comparativo):
    df_quadro = df_quadro_zero.T
    df_quadro.reset_index(inplace=True)
    df_quadro.columns = ["Ref.", "Base Analítica"]
    df_quadro["Contrato/Tela Sistêmica"] = ""

    conferencias = comparativo[comparativo["Contrato"] == contrato].drop_duplicates()[
        status_cols
    ]
    conferencias["Contrato"] = "Conferido"
    conferencias = conferencias.T.reset_index()
    conferencias.columns = ["Ref.", "Base Analítica x Contrato/Tela Sistêmica"]

    df_quadro["Base Analítica x Contrato/Tela Sistêmica"] = conferencias[
        "Base Analítica x Contrato/Tela Sistêmica"
    ]
    df_quadro["Anexo"] = f"Anexo N_A{contrato}"

    df_quadro["Contrato/Tela Sistêmica"] = df_quadro.apply(
        lambda x: (
            x["Base Analítica"]
            if x["Base Analítica x Contrato/Tela Sistêmica"] == "Conferido"
            else ""
        ),
        axis=1,
    )
    df_quadro["Base Analítica x Contrato/Tela Sistêmica"] = df_quadro[
        "Base Analítica x Contrato/Tela Sistêmica"
    ].apply(lambda x: "Divergente" if x != "Conferido" else x)

    df_quadro["Base Analítica"] = df_quadro.apply(
        lambda x: (
            format_values(x["Base Analítica"], moeda=True)
            if "Valor do Bem" in x["Ref."]
            else x["Base Analítica"]
        ),
        axis=1,
    )
    df_quadro["Contrato/Tela Sistêmica"] = df_quadro.apply(
        lambda x: (
            format_values(x["Contrato/Tela Sistêmica"], moeda=True)
            if "Valor do Bem" in x["Ref."]
            else x["Contrato/Tela Sistêmica"]
        ),
        axis=1,
    )

    workbook = anexo_b.create_excel_model(
        file_name="Modelo_Com_Linha_Oculta.xlsx",
        header_color="002060",
        header_text="RESUMO DAS VALIDAÇÕES DAS OPERAÇÕES DE LEASING - ITAÚ UNIBANCO S.A.",
        header_list=[
            "Ref.",
            "Base Analítica",
            "Contrato/Tela Sistêmica",
            "Base Analítica x Contrato/Tela Sistêmica",
            "Anexo",
        ],
        alignments=[
            "center",
            "center",
            "center",
            "center",
            "center",
        ],  # Alinhamentos específicos por coluna
        fill_colors=["002060", "002060", "002060", "002060", "002060"],
        merge_columns=[1, 1, 1, 2, 1],
        font="Calibri",
        font_size=10,
    )

    anexo_b.fill_excel_data(
        workbook,
        worksheet_name="Modelo",
        df=df_quadro,
        text_colors=[None, None, None, None, "FF0000"],
        bold=[False, False, False, False, True],
        cell_borders=[True, True, True, True, True],
        alignments=["left", "center", "center", "right", "center"],
        columns_to_merge=["Anexo"],
        column_types=["texto", "texto", "texto", "texto", "texto"],
    )

    return workbook

#### processar_quadro_um

nao precisa mexer

In [ ]:
def processar_quadro_um(df_contratuais_vs_registro_contabil, contrato):
    df_quadro = df_contratuais_vs_registro_contabil[
        df_contratuais_vs_registro_contabil["Contrato"] == contrato
    ]

    df_quadro["Valor do ativo contabilizado"] = df_quadro[
        "Valor do ativo contabilizado"
    ].apply(lambda x: np.nan if x == "0.00" else x)
    df_quadro["Valores residuais a realizar contabilizado"] = df_quadro[
        "Valores residuais a realizar contabilizado"
    ].apply(lambda x: np.nan if x == "0.00" else x)
    df_quadro["Total Contraprestação de Arrendamento - CPA"] = df_quadro[
        "Total Contraprestação de Arrendamento - CPA"
    ].apply(lambda x: np.nan if x == "0.00" else x)
    df_quadro["Valor do ativo contabilizado"] = df_quadro[
        "Valor do ativo contabilizado"
    ].apply(lambda x: np.nan if x == "0.0000" else x)
    df_quadro["Valores residuais a realizar contabilizado"] = df_quadro[
        "Valores residuais a realizar contabilizado"
    ].apply(lambda x: np.nan if x == "0.0000" else x)
    df_quadro["Total Contraprestação de Arrendamento - CPA"] = df_quadro[
        "Total Contraprestação de Arrendamento - CPA"
    ].apply(lambda x: np.nan if x == "0.0000" else x)

    df_quadro_pivot = df_quadro.melt(
        id_vars=["Contrato", "COSIF"],  # Colunas que permanecerão como estão
        value_vars=[
            "Valor do ativo contabilizado",
            "Valores residuais a realizar contabilizado",
            "Total Contraprestação de Arrendamento - CPA",
        ],  # Colunas a serem transformadas
        var_name="Descrição",  # Nome da nova coluna com os nomes das descrições
        value_name="Valor",  # Nome da nova coluna com os valores
    )
    df_quadro_pivot.dropna(subset=["Valor"], inplace=True)
    # Filtrar apenas as linhas com valores diferentes de zero, se necessário
    df_quadro_pivot = df_quadro_pivot[df_quadro_pivot["Valor"] != 0]
    df_quadro_pivot = df_quadro_pivot[["Descrição", "COSIF", "Valor"]]
    df_quadro_pivot.columns = ["Descrição", "COSIF", "Valor Contabilizado"]
    df_quadro_pivot["Anexo"] = (
        f"B{contrato}"  # f"{name}_DADOS CONTRATUAIS X REGISTRO CONTÁBIL.xlsx"
    )

    df_quadro_pivot["Valor Contabilizado"] = df_quadro_pivot[
        "Valor Contabilizado"
    ].apply(format_values)

    workbook = anexo_b.create_excel_model(
        file_name="Modelo_Com_Linha_Oculta.xlsx",
        header_color="002060",
        header_text=f"DADOS CONTRATUAIS X REGISTRO CONTÁBIL",
        header_list=["Descrição", "COSIF", "Valor Contabilizado", "Anexo"],
        alignments=[
            "center",
            "center",
            "center",
            "center",
        ],  # Alinhamentos específicos por coluna
        fill_colors=["002060", "002060", "002060", "002060"],
        merge_columns=[1, 1, 3, 1],
        font="Calibri",
        font_size=10,
    )

    anexo_b.fill_excel_data(
        workbook,
        "Modelo",
        df_quadro_pivot,
        text_colors=[None, None, None, "FF0000"],
        bold=[False, False, False, True],
        cell_borders=[True, True, True, True],
        cell_alignments=["left", "center", "right", "center"],
        columns_to_merge=["Anexo"],
        column_types=["texto", "texto", "moeda", "texto"],
    )

    return workbook

#### processar_quadro_dois

nao precisa mexer

In [ ]:
def processar_quadro_dois(df_demonstracao_valor_anual, contrato):

    df_quadro = df_demonstracao_valor_anual[df_demonstracao_valor_anual["Contrato"]==contrato]
    
    df_quadro["Anexo"] = f"B{contrato}"
    
    df_quadro.columns = ["Contrato","Ano","COSIF","Valor Líquido Contabilizado","Descrição","Anexo"]
    df_quadro.fillna("",inplace=True)
    df_quadro["Valor Líquido Contabilizado"]=df_quadro["Valor Líquido Contabilizado"].apply(format_values)
    
    workbook = anexo_b.create_excel_model(
        file_name="Modelo_Com_Linha_Oculta.xlsx",
        header_color="002060",
        header_text=f"DEMONSTRAÇÃO DO VALOR ANUAL DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO  - CONTRATO {contrato}",
        header_list=["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"],
        alignments=["center", "center", "center", "center","center"],  # Alinhamentos específicos por coluna
        fill_colors=["002060", "002060","002060","002060","002060"],
        merge_columns=[1, 1, 1, 2, 1],
        font="Calibri",
        font_size=10
    )
    df_quadro = df_quadro[["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"]]
    # Chamar a função para preencher os dados com formatação
    anexo_b.fill_excel_data(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"]], 
                                text_colors=["FF0000", None, None, None, "FF0000"], 
                                bold=[True, False, False, False, True], 
                                cell_borders=[True, True, True, True, True],
                                cell_alignments=["center", "center", "center", "right" , "center"],
                                columns_to_merge=["COSIF","Descrição","Anexo"],
                                column_types=["texto", "texto", "texto", "moeda", "texto"])
    return workbook

#### processar_quadro_tres

In [ ]:
def processar_quadro_tres(df_efeito_resultado_contabil, contrato):
    df_quadro = df_efeito_resultado_contabil[df_efeito_resultado_contabil["Contrato"]==contrato]
    # df_efeito_resultado_contabil_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_RESULTADO CONTÁBIL LÍQUIDO DO CONTRATO COM EFEITO DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_RESULTADO CONTÁBIL LÍQUIDO DO CONTRATO COM EFEITO DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO.xlsx"
    df_quadro.columns = ["Contrato","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Ano","Anexo"]
    df_quadro["Receita de Contraprestação - Inclui Superveniência (A)"]=df_quadro["Receita de Contraprestação - Inclui Superveniência (A)"].apply(format_values)
    df_quadro["Despesa de Depreciação - Inclui Insuficiência (B)"]=df_quadro["Despesa de Depreciação - Inclui Insuficiência (B)"].apply(format_values)
    df_quadro["Descontos Concedidos (C)"]=df_quadro["Descontos Concedidos (C)"].apply(format_values)
    df_quadro["LAIR (A) + (B) + (C) = (D)"]=df_quadro["LAIR (A) + (B) + (C) = (D)"].apply(format_values)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"EFEITOS NO RESULTADO CONTÁBIL RELATIVOS AO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )

    # Chamar a função para preencher os dados com formatação
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

#### processar_quadro_quatro

In [ ]:
def processar_quadro_quatro(df_base_calculo_irpj, contrato):
    df_quadro = df_base_calculo_irpj[df_base_calculo_irpj["Contrato"]==contrato]
    # df_base_calculo_irpj_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"]
    df_quadro["RESULTADO ANTES DO IRPJ"]=df_quadro["RESULTADO ANTES DO IRPJ"].apply(format_values)
    df_quadro["ADIÇÕES -  Descontos Concedidos"]=df_quadro["ADIÇÕES -  Descontos Concedidos"].apply(format_values)
    df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"]=df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"].apply(format_values)
    df_quadro["Base de Cálculo do IRPJ"]=df_quadro["Base de Cálculo do IRPJ"].apply(format_values)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

#### processar_quadro_cinco

In [ ]:
def processar_quadro_cinco(df_base_calculo_csll, contrato):
    df_quadro = df_base_calculo_csll[df_base_calculo_csll["Contrato"]==contrato]
    # df_base_calculo_csll_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"]
    df_quadro["RESULTADO ANTES DA CSLL"]=df_quadro["RESULTADO ANTES DA CSLL"].apply(format_values)
    df_quadro["ADIÇÕES -  Descontos Concedidos"]=df_quadro["ADIÇÕES -  Descontos Concedidos"].apply(format_values)
    df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"]=df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"].apply(format_values)
    df_quadro["Base de Cálculo da CSLL"]=df_quadro["Base de Cálculo da CSLL"].apply(format_values)

    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook 

#### processar_quadro_seis

In [ ]:
def processar_quadro_seis(df_demonstrativo_efeito_nulo, contrato):
    df_quadro = df_demonstrativo_efeito_nulo[df_demonstrativo_efeito_nulo["Contrato"]==contrato]
    # df_demonstrativo_efeito_nulo_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"]
    df_quadro["BASE DO IRPJ"]=df_quadro["BASE DO IRPJ"].apply(format_values)
    df_quadro["BASE DA CSLL"]=df_quadro["BASE DA CSLL"].apply(format_values)
    df_quadro["DIFERENÇA"]=df_quadro["DIFERENÇA"].apply(format_values)
    df_quadro["SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO"]=df_quadro["SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO"].apply(format_values)
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

### criar_folha_rosto

In [ ]:
def criar_folha_rosto( contrato
                     , df_quadro_zero
                     , comparativo
                     , caminho_destino
                     , df_contratuais_vs_registro_contabil
                     , df_demonstracao_valor_anual
                     , df_efeito_resultado_contabil
                     , df_base_calculo_irpj
                     , df_base_calculo_csll
                     , df_demonstrativo_efeito_nulo
                     , caminho_base
                     , replace=False ):
    
    nome_final = f"RESUMO_{contrato}.xlsx"

    if not replace and os.path.isfile(Path.joinpath(caminho_destino,nome_final)):
        return
    os.makedirs(caminho_destino,exist_ok=True)
    
    wb_resumo = processar_quadro_zero(df_quadro_zero, status_cols, contrato, comparativo)
    wb_contratuais_vs_registro_contabil = processar_quadro_um(df_contratuais_vs_registro_contabil, contrato)
    wb_demonstracao_valor_anual = processar_quadro_dois(df_demonstracao_valor_anual, contrato)
    wb_efeito_resultado_contabil = processar_quadro_tres(df_efeito_resultado_contabil, contrato)
    wb_base_calculo_irpj = processar_quadro_quatro(df_base_calculo_irpj, contrato)
    wb_base_calculo_csll = processar_quadro_cinco(df_base_calculo_csll, contrato)
    wb_demonstrativo_efeito_nulo = processar_quadro_seis(df_demonstrativo_efeito_nulo, contrato)
    
    unir_workbooks_em_xlsx(
        lista_wb=[wb_resumo, 
                    wb_contratuais_vs_registro_contabil,
                    wb_demonstracao_valor_anual,
                    wb_efeito_resultado_contabil,
                    wb_base_calculo_irpj,
                    wb_base_calculo_csll,
                    wb_demonstrativo_efeito_nulo
                    ],
        path_arquivo_base=caminho_base,
        destino=caminho_destino,
        nome_novo=nome_final,
        celula_inicial="D9"
    )


### process_anexo_pdfs

In [ ]:
def process_anexo_pdfs(caminho_base, limite_mb=5, contador_inicial=0):
    # Caminho do arquivo .xlsx
    # file_path = "filtro2.xlsx"
    # data = pd.read_excel(file_path)
    # data["Contrato"] = data["Contrato"].astype(str)
    # contratos_invalidos = data["Contrato"].values.tolist()
    contratos_invalidos = []  # Zerado conforme o código original
    limite_bytes = limite_mb * 1024 * 1024
    lote = os.path.basename(os.path.dirname(caminho_base))
    anexo = os.path.basename(caminho_base)

    arquivos_info = []
    for f in os.listdir(caminho_base):
        if f.endswith(".pdf"):
            caminho_pdf = os.path.join(caminho_base, f)
            tamanho = os.path.getsize(caminho_pdf)
            # Verifica se o número do contrato (apenas dígitos do nome) não está na lista de contratos inválidos
            if re.sub(r'\D', '', f) not in contratos_invalidos:
                arquivos_info.append((re.sub(r'\D', '', f), caminho_pdf, tamanho))

    # Cria a pasta de saída para os lotes
    pasta_final = f"{caminho_base}_{lote}_partes"
    os.makedirs(pasta_final, exist_ok=True)

    # Função para dividir PDFs que excedem o tamanho máximo permitido
    def dividir_pdf(origem, tamanho_max, label):
        partes = []
        with open(origem, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            writer = None
            parte_indice = 1
            tamanho_acumulado = 0
            for page in reader.pages:
                temp_writer = PyPDF2.PdfWriter()
                temp_writer.add_page(page)
                # Salva temporariamente para verificar o tamanho da página
                with open("temp_part.pdf", "wb") as temp_out:
                    temp_writer.write(temp_out)
                parte_tamanho = os.path.getsize("temp_part.pdf")
                if not writer or (tamanho_acumulado + parte_tamanho > tamanho_max):
                    if writer:
                        nome_parte = os.path.join(pasta_final, f"{label}_parte_{parte_indice}.pdf")
                        with open(nome_parte, "wb") as out:
                            writer.write(out)
                        partes.append(nome_parte)
                        parte_indice += 1
                    writer = PyPDF2.PdfWriter()
                    tamanho_acumulado = 0
                writer.add_page(page)
                tamanho_acumulado += parte_tamanho
            if writer:
                nome_parte = os.path.join(pasta_final, f"{label}_parte_{parte_indice}.pdf")
                with open(nome_parte, "wb") as out:
                    writer.write(out)
                partes.append(nome_parte)
        os.remove("temp_part.pdf")
        return partes

    # Agrupa os PDFs (ou partes deles) em grupos cujo tamanho total não exceda 'limite_bytes'
    grupos = []
    grupo_atual = []
    tamanho_atual = 0
    for pasta, caminho_pdf, tamanho in arquivos_info:
        if tamanho > limite_bytes:
            nome_base = os.path.splitext(os.path.basename(caminho_pdf))[0]
            partes = dividir_pdf(caminho_pdf, limite_bytes, nome_base)
            for parte in partes:
                p_size = os.path.getsize(parte)
                parte_nome = os.path.basename(parte)
                if p_size + tamanho_atual <= limite_bytes:
                    grupo_atual.append((pasta, parte_nome))
                    tamanho_atual += p_size
                else:
                    if grupo_atual:
                        grupos.append(grupo_atual)
                    grupo_atual = [(pasta, parte_nome)]
                    tamanho_atual = p_size
        else:
            if tamanho + tamanho_atual <= limite_bytes:
                grupo_atual.append((pasta, caminho_pdf))
                tamanho_atual += tamanho
            else:
                if grupo_atual:
                    grupos.append(grupo_atual)
                grupo_atual = [(pasta, caminho_pdf)]
                tamanho_atual = tamanho
    if grupo_atual:
        grupos.append(grupo_atual)

    dados_excel = []
    final_pdfs = []  # Lista para armazenar os caminhos dos PDFs finais (lotes)
    for i, grupo in enumerate(tqdm(grupos, desc="Processando grupos"), start=1):
        # Registra informações para o Excel (sem criar o TXT)
        for (p, arquivo) in grupo:
            parte = f" - Parte {arquivo.split('_parte_')[-1].split('.pdf')[0]}" if "_parte_" in arquivo else ""
            dados_excel.append({"Contrato Número": p, "Lote": f"{anexo}_{lote}_parte_{i}.pdf", "Parte do contrato": parte})

        # Cria o PDF final para o grupo
        escritor_final = PyPDF2.PdfWriter()
        for (p, arquivo) in grupo:
            print(p)
            # Cria uma página de cabeçalho com o número do contrato
            buffer = io.BytesIO()
            c = canvas.Canvas(buffer, pagesize=letter)
            c.setFont("Helvetica-Bold", 14)
            width, height = letter
            texto = f"Contrato Número: {p}"
            print(texto)
            c.drawCentredString(width / 2, height / 2, texto)
            c.showPage()
            c.save()
            buffer.seek(0)

            leitor_temp = PyPDF2.PdfReader(buffer)
            for pagina_temp in leitor_temp.pages:
                escritor_final.add_page(pagina_temp)

            # Determina o caminho completo do PDF a ser mesclado
            if os.path.exists(arquivo):
                caminho_completo = arquivo
            else:
                caminho_completo = os.path.join(pasta_final, arquivo)

            with open(caminho_completo, "rb") as pdf_in:
                leitor = PyPDF2.PdfReader(pdf_in)
                for pagina in leitor.pages:
                    escritor_final.add_page(pagina)

        nome_pdf_final = os.path.join(pasta_final, f"{anexo}_{lote}_parte_{i}.pdf")
        with open(nome_pdf_final, "wb") as saida:
            escritor_final.write(saida)
        final_pdfs.append(nome_pdf_final)

    # Agrupa os PDFs finais em subpastas: cada subpasta terá, no máximo, 14 documentos e tamanho total <= 140 MB.
    # Cria um mapeamento de cada PDF final para a sua subpasta.
    pdf_to_subfolder = {}
    max_docs = 14
    max_folder_size = 140 * 1024 * 1024  # 140 MB em bytes
    current_group_files = []
    current_group_size = 0
    folder_index = contador_inicial

    for pdf_file in final_pdfs:
        file_size = os.path.getsize(pdf_file)
        if (len(current_group_files) >= max_docs) or (current_group_size + file_size > max_folder_size):
            subfolder = os.path.join(pasta_final, f"{folder_index}_{lote}_grupo")
            os.makedirs(subfolder, exist_ok=True)
            for f_pdf in current_group_files:
                destino = os.path.join(subfolder, os.path.basename(f_pdf))
                os.rename(f_pdf, destino)
                pdf_to_subfolder[os.path.basename(f_pdf)] = f"{folder_index}_{lote}_grupo"
            folder_index += 1
            current_group_files = []
            current_group_size = 0
        current_group_files.append(pdf_file)
        current_group_size += file_size

    if current_group_files:
        subfolder = os.path.join(pasta_final, f"{folder_index}_{lote}_grupo")
        os.makedirs(subfolder, exist_ok=True)
        for f_pdf in current_group_files:
            destino = os.path.join(subfolder, os.path.basename(f_pdf))
            os.rename(f_pdf, destino)
            pdf_to_subfolder[os.path.basename(f_pdf)] = f"{folder_index}_{lote}_grupo"

    # Cria o arquivo Excel consolidado com os dados dos contratos e lotes,
    # incluindo a coluna "Pasta Interna" que indica a subpasta onde o PDF foi salvo.
    df = pd.DataFrame(dados_excel)
    df["Pasta Interna"] = df["Lote"].apply(lambda lote: pdf_to_subfolder.get(lote, ""))
    df.to_excel(os.path.join(str(caminho_base) + f"_{lote}_partes", f"{anexo}_{lote}_partes.xlsx"), index=False)

    # Remove arquivos temporários
    temp_files = ["temp_part.pdf"]
    for temp_file in temp_files:
        if os.path.exists(temp_file):
            os.remove(temp_file)

    # Compacta as pastas criadas em arquivos zip
    for p in glob.glob(str(caminho_base) + f"_{lote}_partes/*.pdf"):
        os.remove

    shutil.make_archive(str(caminho_base) + f"_{lote}_partes", 'zip', str(caminho_base) + f"_{lote}_partes")
    shutil.rmtree(str(caminho_base) + f"_{lote}_partes")
    return folder_index+1

## Processamento

In [ ]:
tipos_colunas = {
    "n° do contrato": str,
    "Razão Social_Nome Completo do Arrendatário": str,
    "CNPJ_CPF do Arrendatário": str,
    "Nº de Parcelas Contratadas": str,
    "Valor do Bem": float,
}
data_columns = ["data do contrato", "Liquidação"]

df_base = pd.read_excel(path_base, dtype=tipos_colunas)
df_base["N° de parcelas contratadas"] = (
    pd.to_numeric(df_base["N° de parcelas contratadas"], errors="coerce")
    .fillna(0)
    .astype(int)
)

df_base["CNPJ_CPF do Arrendatário"] = df_base["CNPJ_CPF do Arrendatário"].apply(
    formatar_cpf_cnpj
)

df_base["Razão Social_Nome Completo do Arrendatário"] = df_base[
    "Razão Social_Nome Completo do Arrendatário"
].apply(to_pascal_case)

# Exemplo de uso
df_base = format_date_columns(df_base, data_columns)
df_base["n° do contrato"] = df_base["n° do contrato"].astype(str)

In [ ]:
df_base = df_base[
    [
        "n° do contrato",
        "Razão Social_Nome Completo do Arrendatário",
        "CNPJ_CPF do Arrendatário",
        "Valor do Bem",
        "data do contrato",
        "Liquidação",
        "N° de parcelas contratadas",
    ]
]
df_base.columns = [
    "Nº do Contrato",
    "Cliente",
    "CNPJ/CPF",
    "Valor do Bem",
    "Data do Contrato",
    "Data de Liquidação",
    "Quantidade de Parcelas",
]

In [ ]:
import threading
import traceback
import sys
from tqdm import tqdm


def safe_criar_folha_rosto(
    contrato,
    info,
    comparativo,
    caminho_destino,
    df_contratuais_vs_registro_contabil,
    df_demonstracao_valor_anual,
    df_efeito_resultado_contabil,
    df_base_calculo_irpj,
    df_base_calculo_csll,
    df_demonstrativo_efeito_nulo,
    caminho_base,
    replace,
):
    try:
        criar_folha_rosto(
            contrato,
            info,
            comparativo,
            caminho_destino,
            df_contratuais_vs_registro_contabil,
            df_demonstracao_valor_anual,
            df_efeito_resultado_contabil,
            df_base_calculo_irpj,
            df_base_calculo_csll,
            df_demonstrativo_efeito_nulo,
            caminho_base,
            replace,
        )
    except Exception as e:
        # imprime nome do grupo, mensagem de erro e traceback completo
        print(f"[ERRO] no grupo {contrato}: {e}", file=sys.stderr)
        # traceback.print_exc()

# coisa

In [ ]:
contratos_validos = comparativo["Contrato"].values.tolist()

In [ ]:
df_base = df_base[df_base["Nº do Contrato"].isin(contratos_validos)]
info_contratos = df_base.groupby("Nº do Contrato")
replace = True

In [ ]:
warnings.filterwarnings("ignore")
thread_limit = 30
idx = 0
threads = []

for contrato, info in tqdm(
    info_contratos, desc="Processando grupos", total=len(info_contratos)
):
    thread = threading.Thread(
        target=safe_criar_folha_rosto,
        args=(
            contrato,
            info,
            comparativo,
            output_excel,
            df_contratuais_vs_registro_contabil,
            df_demonstracao_valor_anual,
            df_efeito_resultado_contabil,
            df_base_calculo_irpj,
            df_base_calculo_csll,
            df_demonstrativo_efeito_nulo,
            caminho_base,
            replace,
        ),
    )
    thread.start()
    threads.append(thread)

    if len(threads) >= thread_limit or idx == len(info_contratos) - 1:
        # Aguarde até que todas as threads terminem
        for thread in threads:
            thread.join()
        threads = []
    idx += 1